In [27]:
import pandas as pd

In [28]:
df=pd.read_csv("../input/fake-news/train.csv")

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
df=df.dropna()

In [31]:
df.describe()

,id,label
count,18285.000000,18285.000000
mean,10409.807383,0.433361
std,5998.216236,0.495553
min,0.000000,0.000000
25%,5217.000000,0.000000
50%,10422.000000,0.000000
75%,15603.000000,1.000000
max,20799.000000,1.000000


In [32]:
X=df.drop('label',axis=1)

In [33]:
Y=df['label']

In [34]:
X.shape

(18285, 4)

In [35]:
Y.shape

(18285,)

In [36]:
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [37]:
voc_size=5000

In [38]:
Train=X.copy()

In [39]:
Train['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [40]:
Train.reset_index(inplace=True)
#because we had drop NaN values

In [41]:
Train

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [42]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

**Data Preprocessing**

In [43]:
ps = PorterStemmer()
corpus=[]
for i in range(0, len(Train)):
    review = re.sub('[^a-zA-Z]', ' ', Train['title'][i])
    review=review.lower()
    review=review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [44]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]


In [45]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4997 3073 3219]
 [   0    0    0 ... 3963 1626 3471]
 [   0    0    0 ... 4392  354 2299]
 ...
 [   0    0    0 ...  692  809  896]
 [   0    0    0 ...  389 4502 1262]
 [   0    0    0 ... 1534 3924 3448]]


In [46]:
len(embedded_docs)

18285

**Creating Model**

In [49]:
embedding_vect_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vect_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(Y)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [53]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)


Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3354 - accuracy: 0.8423 - val_loss: 0.2062 - val_accuracy: 0.9142
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1388 - accuracy: 0.9468 - val_loss: 0.1907 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0992 - accuracy: 0.9648 - val_loss: 0.2429 - val_accuracy: 0.9176
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0679 - accuracy: 0.9771 - val_loss: 0.2763 - val_accuracy: 0.9142
Epoch 5/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0431 - accuracy: 0.9842 - val_loss: 0.2561 - val_accuracy: 0.9092
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0299 - accuracy: 0.9908 - val_loss: 0.4110 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0169 - accuracy: 0.9944 - val_loss: 0.4326 - val_accuracy: 0.9112

**Loading our Test Dataset for prediction**

In [54]:
Test=pd.read_csv("../input/fake-news/test.csv")

In [55]:
Test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [56]:
Test_id = Test["id"]

In [57]:
Test_id

0       20800
1       20801
2       20802
3       20803
4       20804
        ...  
5195    25995
5196    25996
5197    25997
5198    25998
5199    25999
Name: id, Length: 5200, dtype: int64

In [58]:
# here we are removing these columns as they are not so important
Test=Test.drop(['text','author','id'],axis=1)



In [59]:
Test.head()

,title
0,"Specter of Trump Loosens Tongues, if Not Purse..."
1,Russian warships ready to strike terrorists ne...
2,#NoDAPL: Native American Leaders Vow to Stay A...
3,"Tim Tebow Will Attempt Another Comeback, This ..."
4,Keiser Report: Meme Wars (E995)


In [60]:
Test.isnull().sum()

title    122
dtype: int64

In [61]:
Test.fillna('fake fake fake', inplace=True)
# as the solution file that we have to submitted in kaggle expects 
# it to have 5200 rows so we can't drop rows in the test dataset

In [62]:
Test.shape

(5200, 1)

In [65]:
# here we are creating corpus for the test dataset
ps=PorterStemmer()
corpus_test=[]
for i in range(0,len(Test)):
    review = re.sub('[^a-zA-Z]', " ",Test['title'][i] )
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = " ".join(review)
    corpus_test.append(review)

In [66]:
onehot_Test = [one_hot(words,voc_size) for words in corpus_test]

In [67]:
sentence_length = 20
embedded_docs_test = pad_sequences(onehot_Test, padding='pre', maxlen=sentence_length)
print(embedded_docs_test)

[[   0    0    0 ...  692  809  896]
 [   0    0    0 ... 3459 2860 2273]
 [   0    0    0 ...  139 2832 2589]
 ...
 [   0    0    0 ...  692  809  896]
 [   0    0    0 ... 4841 3597 4505]
 [   0    0    0 ...  692  809  896]]


In [68]:
x_test=np.array(embedded_docs_test)

Predictions

In [69]:
predicted_value = model.predict_classes(x_test)

# Submission

In [70]:
val=[]
for i in predicted_value:
    val.append(i[0])


In [72]:
submission = pd.DataFrame({'id':Test_id, 'label':val})
submission.shape

(5200, 2)

In [73]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,0
4,20804,1


In [76]:
submission.to_csv('submission.csv',index=False)

NameError: name 'submit_sample' is not defined

In [75]:
submit_sample.to_csv('submission.csv',index=False)

NameError: name 'submit_sample' is not defined